In [1]:
from ftx_client import FtxClient
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import supertrend as spt

from config import API_KEY, API_SECRET
from pprint import pprint
import numpy as np
import time
from telegram_api_manager import TelegramAPIManager
import json
import os
import sys
import logging

plt.ioff()
FIGURE_PATH = "figures"

In [2]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
perpetuals = [
    "ETH-PERP", 
    "MATIC-PERP", 
    "SOL-PERP", 
    "DOGE-PERP", 
    "ADA-PERP", 
    "RUNE-PERP", 
    "CHZ-PERP", 
    "SUSHI-PERP",
    "DOT-PERP",
    "FTT-PERP",
    "STEP-PERP",
    "BTT-PERP",
    "FIL-PERP",
]

In [ ]:
trades = {}
for perp in perpetuals:
    trades[perp] = []

tapi = TelegramAPIManager(group=False)

while(True):
    # Open new FTX Session
    ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

    for perp in perpetuals:
        df = ftx.get_historical_market_data(perp, interval="1h", start_time="10 days ago")

        # Perform supertrend analysis
        df["st"], df["upt"], df["dt"] = spt.supertrend_analysis(df.high, df.low, df.close, look_back=10, multiplier=3)
        df["long_trig"], df["short_trig"], df["st_signal"] = spt.get_supertrend_signals(df.close, df.st)
        
        # Check 200 EMA for confirmation
        df["ema200"] = spt.calculate_ema(df, time_period=200)
        
        figure_path = spt.plot_and_save_figure(perp, df)
        
        # Set precision for orders
        precision = len(str(df.close[0]).split(".")[1])
        
        # Take profit calculator and stop loss
        stop_loss = round(df.st[-1], precision)
        long_profit_10pct, short_profit_10pct = spt.take_profit_calc(df.close[-1], profit_percent=10, precision=precision)
        long_profit_5pct, short_profit_5pct = spt.take_profit_calc(df.close[-1], profit_percent=5, precision=precision)
        
        # Check last element of signal array
        last_signal = df.st_signal[-1]
        if last_signal != 0:
            # Set up dict for new position
            new_position = {
                "market": perp, 
                "entry": df.close[-1], 
                "stop_loss": stop_loss,
            }
            
            if last_signal == 1:
                # Long position
                new_position["side"] = "buy"
                new_position["10pctprofit"] = long_profit_10pct
                new_position["5pctprofit"] = long_profit_5pct
            elif last_signal == -1:
                # Short position
                new_position["side"] = "sell"
                new_position["10pctprofit"] = short_profit_10pct
                new_position["5pctprofit"] = short_profit_5pct

            if df.close[-1] < df.ema200[-1]:
                new_position["ema200"] = "under"
            else:
                new_position["ema200"] = "over"
            
            new_position["stoch_rsi"] = spt.calculate_stoch_rsi(df)
            
            logger.info(new_position)
            trades[perp].append(new_position)
            tapi.send_photo(figure_path, caption=json.dumps(new_position, indent=2, default=str))
            
            # Close position if needed
            open_position = ftx.check_open_position(perp)
            if open_position:
                if open_position["side"] != new_position["side"]:
                    success, response = market_close_and_cancel_orders(
                                            perp, 
                                            side=new_position["side"], 
                                            size=open_position["size"]
                                        )
                    
                    if success:
                        close_position_text = f"({perp}) Position closed at {response['price']}"
                    else:
                        close_position_text = f"({perp}) Position failed to close: Message {response}"
                    
                    tapi.send_photo(figure_path, caption=close_position_text)
        else:
            pass
        
        # Update stop loss on open position
        if ftx.check_open_position(perp):
            updated_stop_loss = ftx.update_stop_loss(market=perp, stop_loss=stop_loss)
            updated_sl_text = f"({perp}) Updated stop loss = {updated_stop_loss} USD"
            logger.info(updated_sl_text)
            tapi.send_photo(figure_path, caption=updated_sl_text)

    logger.info("Sleeping for 60 mins")
    ftx._session.close()
    time.sleep(60 * 60)

2021-07-15 16:05:03,911 - telegram_api_manager - INFO - cryptobot (@pymparoobot) is now running using Telegram personal chat id
2021-07-15 16:05:29,989 - __main__ - INFO - Sleeping for 60 mins


In [ ]:
calculate_pnl(perpetuals, trades, 10)